In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from pycaret.classification import *

from glob import glob

import os
import pandas as pd
import numpy as np

In [2]:
pwd = os.getcwd()

genre_path = glob(f'{pwd}\\*genre.csv', recursive=True)[0]
data_path = glob(f'{pwd}\\*data.csv', recursive=True)[0]
tag_path = glob(f'{pwd}\\tag*.csv', recursive=True)[0]

print(genre_path, data_path, tag_path)

c:\Users\reot1\OneDrive\바탕 화면\ml2\EDA\LSH\model\one_hot_genre.csv c:\Users\reot1\OneDrive\바탕 화면\ml2\EDA\LSH\model\pre_data.csv c:\Users\reot1\OneDrive\바탕 화면\ml2\EDA\LSH\model\tag_merge.csv


In [3]:
g_df = pd.read_csv(genre_path)
r_df = pd.read_csv(data_path)
t_df = pd.read_csv(tag_path)

In [4]:
merge_df = pd.merge(r_df, g_df).merge(t_df, on='appid')
# merge_df = pd.merge(r_df, g_df)
merge_df.head(1)

,appid,24_Hour_Peak,All_time_peak,positive,negative,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,num_lang,required_age,is_free,controller_support,dlc,metacritic,metacritic_tf,windows,mac,linux,platforms_num,recommendations,achievements,days_after_release,Review,year,month,Target_,Indie_x,Animation&Modeling,MassivelyMultiplayer,N,WebPublishing,PhotoEditing,Adventure_x,RPG_x,Racing_x,Gore_x,Sports_x,Utilities_x,Accounting,AudioProduction,Movie_x,Short_x,Nudity_x,Tutorial_x,Documentary_x,Education_x,ActionGames,SexualContent,FreetoPlay,Episodic_x,VideoProduction,Design&Illustration,Violent_x,EarlyAccess,Casual_x,Action_x,Strategy_x,360V,SoftwareTraining,GameDevelopment,Simulation_x,cluster,genre_set,Cartoon,Hunting,Time Management,Zombies,Isometric,Chess,Musou,Experimental,3D,Walking Simulator,Stealth,Parody,Bullet Time,Skating,Building,Mystery Dungeon,8-bit Music,Farming,Philosophical,Real Time Tactics,RPG_y,Platformer,World War II,Cozy,Difficult,Football (Soccer),Party-Based RPG,Twin Stick Shooter,Trains,Parkour,Golf,Politics,Snow,Female Protagonist,Benchmark,Auto Battler,Tutorial_y,Crime,Modern,Political Sim,Space,Hardware,Motocross,Score Attack,Coding,Minimalist,Martial Arts,Addictive,Level Editor,Conspiracy,Cooking,Tabletop,Fox,Souls-like,Simulation_y,Dark Comedy,Looter Shooter,Tennis,Time Manipulation,Artificial Intelligence,Warhammer 40K,Spelling,Audio Production,Dark,Fantasy,Hacking,Life Sim,Tanks,Visual Novel,Alternate History,Battle Royale,Touch-Friendly,Job Simulator,Superhero,Open World,Pool,Feature Film,Split Screen,Historical,Racing_y,Wargame,Emotional,VR,Sokoban,LEGO,4 Player Local,NSFW,Survival,Hockey,Tower Defense,Comedy,Sailing,Massively Multiplayer,RPGMaker,Werewolves,Mining,Action Roguelike,Music,Controller,Narration,Cycling,Bikes,Volleyball,Escape Room,Military,Bowling,Intentionally Awkward Controls,Bullet Hell,Action RPG,War,Software Training,Mahjong,Cyberpunk,Psychedelic,Party Game,3D Platformer,Boxing,ATV,Masterpiece,Board Game,Mystery,Documentary_y,Side Scroller,Team-Based,Exploration,Perma Death,Hero Shooter,Action_y,Violent_y,America,Remake,Creature Collector,Boss Rush,Utilities_y,Underground,Fighting,Turn-Based Combat,Moddable,Political,Sniper,Conversation,Archery,Hand-drawn,Rhythm,Asynchronous Multiplayer,Dark Humor,Asymmetric VR,Typing,Card Battler,Combat,Survival Horror,Psychological,Wholesome,Flight,Motorbike,Education_y,Drama,Puzzle,Jump Scare,Automobile Sim,Spaceships,Assassin,Silent Protagonist,Fishing,First-Person,Voxel,Steampunk,Animation & Modeling,Satire,Gore_y,Nudity_y,Mouse only,Local Multiplayer,Traditional Roguelike,Hex Grid,Ambient,Colony Sim,Post-apocalyptic,Spectacle fighter,Anime,Idler,MOBA,Dragons,Gambling,...,Gothic,Choices Matter,On-Rails Shooter,Blood,Social Deduction,Mod,Steam Machine,Foreign,Realistic,Photo Editing,Skiing,Destruction,Immersive Sim,Third-Person Shooter,Time Travel,Instrumental Music,Loot,Cartoony,Epic,Game Development,Trading Card Game,Immersive,Robots,Precision Platformer,1980s,Comic Book,Soccer,Adventure_y,Dating Sim,Dungeon Crawler,Rugby,Cinematic,Mythology,Western,4X,Heist,Base-Building,Great Soundtrack,Outbreak Sim,World War I,Football,Open World Survival Craft,Thriller,Noir,Class-Based,Snooker,Based On A Novel,Jet,CRPG,Word Game,Football (American),Lore-Rich,Crafting,Beautiful,Turn-Based Tactics,PvP,Card Game,FPS,Sci-fi,Pirates,Interactive Fiction,Voice Control,Vikings,Funny,Match 3,Rome,Vehicular Combat,Transportation,Trivia,Transhumanism,City Builder,Nostalgia,Top-Down,Villain Protagonist,Management,TrackIR,6DOF,2.5D,Abstract,Cold War,Gun Customization,Early Access,Arena Shooter,Hack and Slash,Crowdfunded,e-sports,Short_y,Dinosaurs,Logic,Romance,Psychological Horror,Economy,Action RTS,2D Platformer,Electronic,Multiple Endings,Soundtrack,Clicker,Music-Based Procedural Generation,Dystopian,Mature,Puzzle-Platformer,Mechs,Shooter,Snowboarding,Mars,Deckbuilding,Sandbox,Minigames,Farming Sim,Vampire,Stylized,Offroad,3D Fighter

In [5]:
merge_df['genre_num'] = merge_df['genre_set'].apply(lambda x:len(set(eval(x))))

In [6]:
merge_copy = merge_df.copy()

In [7]:
# X = merge_copy.drop(['appid','Review', 'Target_'], axis=1)
X = merge_copy.drop(['appid','Review', 'Target_', 'positive', 'genre_set' ,'negative',
                    'recommendations'], axis=1)
y = merge_copy['Target_']

In [8]:
print(X.shape, y.shape)

(46252, 506) (46252,)


In [9]:
X_resampled, y_resampled = SMOTE(random_state=1234,
                                sampling_strategy='all', n_jobs=-1).fit_resample(X, y)

print(X_resampled.shape, y_resampled.shape)

(77676, 506) (77676,)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, 
                                                    test_size=0.3,
                                                    random_state=1234,
                                                    stratify=y_resampled)

In [10]:
X_ = X_resampled.copy()
X_['Target'] = y_resampled

In [11]:
train, test = train_test_split(X_,
                            test_size=0.3,
                            random_state=1234,
                            stratify=X_['Target'])
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [12]:
exp_clf = setup(data = train, target = 'Target', session_id=1234)

,Description,Value
0,session_id,1234
1,Target,Target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(54373, 507)"
5,Missing Values,False
6,Numeric Features,462
7,Categorical Features,44
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
# best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7694,0.9275,0.7697,0.7740,0.7666,0.6925,0.6958,3.9490
et,Extra Trees Classifier,0.7357,0.9089,0.7360,0.7388,0.7331,0.6476,0.6501,7.9760
xgboost,Extreme Gradient Boosting,0.7298,0.9170,0.7300,0.7300,0.7264,0.6398,0.6419,34.6170
lightgbm,Light Gradient Boosting Machine,0.7218,0.9136,0.7220,0.7210,0.7178,0.6290,0.6311,2.7890
gbc,Gradient Boosting Classifier,0.6436,0.8686,0.6438,0.6411,0.6358,0.5248,0.5287,28.5800
dt,Decision Tree Classifier,0.6161,0.7449,0.6164,0.6134,0.6145,0.4882,0.4883,0.9240
knn,K Neighbors Classifier,0.6078,0.8308,0.6084,0.6050,0.5799,0.4772,0.4884,12.9420
ada,Ada Boost Classifier,0.5577,0.7984,0.5578,0.5551,0.5491,0.4102,0.4140,1.5520
ridge,Ridge Classifier,0.5425,0.0000,0.5427,0.5358,0.5203,0.3900,0.3994,0.1560
lda,Linear Discriminant Analysis,0.5420,0.7951,0.5421,0.5432,0.5350,0.3893,0.3936,2.0830


In [13]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7683,0.9249,0.7686,0.7710,0.7656,0.6911,0.6937
1,0.7641,0.9265,0.7644,0.7692,0.7619,0.6854,0.6884
2,0.7643,0.9229,0.7646,0.7720,0.7619,0.6858,0.6901
3,0.7767,0.9293,0.7770,0.7812,0.7736,0.7022,0.7058
4,0.7651,0.9231,0.7654,0.7695,0.7630,0.6868,0.6895
5,0.7583,0.9252,0.7586,0.7614,0.7541,0.6777,0.6814
6,0.7754,0.9286,0.7756,0.7807,0.7731,0.7005,0.7036
7,0.7754,0.9301,0.7756,0.7806,0.7719,0.7005,0.7046
8,0.7730,0.9317,0.7732,0.7762,0.7704,0.6973,0.7001


In [14]:
et = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7308,0.9078,0.7311,0.7318,0.7274,0.6410,0.6432
1,0.7383,0.9115,0.7386,0.7432,0.7358,0.6511,0.6541
2,0.7328,0.9071,0.7331,0.7388,0.7305,0.6437,0.6470
3,0.7472,0.9117,0.7476,0.7519,0.7447,0.6630,0.6659
4,0.7210,0.9047,0.7212,0.7219,0.7182,0.6280,0.6298
5,0.7336,0.9042,0.7339,0.7342,0.7300,0.6448,0.6470
6,0.7336,0.9088,0.7339,0.7377,0.7314,0.6448,0.6472
7,0.7430,0.9120,0.7433,0.7460,0.7400,0.6574,0.6602
8,0.7394,0.9109,0.7396,0.7424,0.7377,0.6525,0.6544


In [15]:
xgb = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7263,0.9171,0.7265,0.7248,0.7221,0.6351,0.6371
1,0.7225,0.9101,0.7228,0.7242,0.7200,0.6301,0.6321
2,0.7270,0.9165,0.7272,0.7278,0.7237,0.6360,0.6384
3,0.7331,0.9190,0.7333,0.7340,0.7292,0.6441,0.6468
4,0.7249,0.9127,0.7251,0.7259,0.7219,0.6332,0.6352
5,0.7302,0.9176,0.7304,0.7292,0.7259,0.6402,0.6425
6,0.7365,0.9193,0.7367,0.7362,0.7333,0.6486,0.6505
7,0.7370,0.9203,0.7372,0.7372,0.7332,0.6493,0.6517
8,0.7344,0.9200,0.7345,0.7334,0.7314,0.6458,0.6473


In [16]:
lgb = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7147,0.9121,0.7150,0.7123,0.7099,0.6197,0.6217
1,0.7147,0.9072,0.7149,0.7154,0.7111,0.6196,0.6218
2,0.7144,0.9127,0.7146,0.7142,0.7107,0.6192,0.6213
3,0.7260,0.9149,0.7262,0.7252,0.7211,0.6346,0.6373
4,0.7099,0.9095,0.7101,0.7089,0.7060,0.6133,0.6152
5,0.7191,0.9143,0.7193,0.7178,0.7149,0.6255,0.6277
6,0.7333,0.9153,0.7335,0.7328,0.7295,0.6444,0.6465
7,0.7307,0.9163,0.7309,0.7306,0.7266,0.6409,0.6434
8,0.7296,0.9179,0.7298,0.7287,0.7268,0.6395,0.6408


In [17]:
tuned_rf = tune_model(rf, optimize='Recall')
tuned_et = tune_model(et, optimize='Recall')
tuned_xgb = tune_model(xgb, optimize='Recall')
tuned_lgb = tune_model(lgb, optimize='Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7465,0.9261,0.7468,0.7464,0.7441,0.6620,0.6634
1,0.7438,0.9200,0.7441,0.7460,0.7425,0.6584,0.6599
2,0.7522,0.9278,0.7525,0.7534,0.7506,0.6696,0.6710
3,0.7543,0.9271,0.7546,0.7541,0.7524,0.6724,0.6736
4,0.7465,0.9264,0.7467,0.7468,0.7446,0.6619,0.6631
5,0.7530,0.9271,0.7533,0.7526,0.7502,0.6707,0.6723
6,0.7556,0.9279,0.7559,0.7560,0.7536,0.6742,0.6755
7,0.7538,0.9280,0.7540,0.7539,0.7520,0.6718,0.6729
8,0.7501,0.9310,0.7503,0.7501,0.7492,0.6668,0.6674


In [33]:
tuned_rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=7, min_weight_fraction_leaf=0.0,
                       n_estimators=160, n_jobs=-1, oob_score=False,
                       random_state=1234, verbose=0, warm_start=False)

In [34]:
rt_final = finalize_model(tuned_rf)
rf_prediction = predict_model(rt_final, data=test)
print(classification_report(rf_prediction, test['Target']))

In [ ]:
et_final = finalize_model(tuned_et)
et_prediction = predict_model(et_final, data=test)
print(classification_report(et_prediction, test['Target']))

In [ ]:
xgb_final = finalize_model(tuned_xgb)
xgb_prediction = predict_model(xgb_final, data=test)
print(classification_report(xgb_prediction, test['Target']))

In [ ]:
lgb_final = finalize_model(tuned_lgb)
lgb_prediction = predict_model(lgb_final, data=test)
print(classification_report(lgb_prediction, test['Target']))

In [11]:
rf = RandomForestClassifier(random_state=1234,n_jobs=-1)
rf.fit(x_train, y_train)
pred_rf = rf.predict(x_test)

print(classification_report(pred_rf, y_test))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76      7139
           1       0.75      0.86      0.80      5097
           2       0.97      0.94      0.95      5959
           3       0.64      0.73      0.68      5108

    accuracy                           0.80     23303
   macro avg       0.80      0.80      0.80     23303
weighted avg       0.81      0.80      0.80     23303



In [12]:
imp_rf_df = pd.DataFrame({'col':rf.feature_names_in_, 'imp':rf.feature_importances_})
imp_rf_df = imp_rf_df.sort_values(by='imp',ascending=False)
imp_rf_df.reset_index(drop=True, inplace=True)
imp_rf_df.head(15)

,col,imp
0,days_after_release,0.055793
1,All_time_peak,0.033963
2,achievements,0.033700
3,initialprice,0.030211
4,month,0.028223
5,price,0.026901
6,Indie_y,0.022577
7,num_lang,0.020449
8,cluster,0.019182
9,Casual_y,0.017579


In [13]:
xgb = XGBClassifier(n_jobs=-1, random_state=1234)
xgb.fit(x_train, y_train)
pred_xgb = xgb.predict(x_test)

print(classification_report(pred_xgb, y_test))

              precision    recall  f1-score   support

           0       0.82      0.69      0.75      6886
           1       0.62      0.75      0.68      4840
           2       0.89      0.84      0.87      6150
           3       0.65      0.69      0.67      5427

    accuracy                           0.74     23303
   macro avg       0.74      0.74      0.74     23303
weighted avg       0.76      0.74      0.75     23303



In [14]:
imp_xgb_df = pd.DataFrame({'col':xgb.feature_names_in_, 'imp':xgb.feature_importances_})
imp_xgb_df = imp_xgb_df.sort_values(by='imp',ascending=False)
imp_xgb_df.reset_index(drop=True, inplace=True)
imp_xgb_df.head(15)

,col,imp
0,controller_support,0.034447
1,mac,0.017992
2,All_time_peak,0.015422
3,Casual_x,0.014610
4,Simulation_x,0.011371
5,dlc,0.010698
6,N,0.009829
7,Indie_x,0.009725
8,Strategy_x,0.008482
9,year,0.008340


In [15]:
lgbm = LGBMClassifier(random_state=1234, n_jobs=-1)
lgbm.fit(x_train, y_train)
pred_lgbm = lgbm.predict(x_test)

print(classification_report(pred_lgbm, y_test))

              precision    recall  f1-score   support

           0       0.81      0.69      0.74      6842
           1       0.60      0.73      0.66      4775
           2       0.88      0.83      0.85      6168
           3       0.65      0.68      0.67      5518

    accuracy                           0.73     23303
   macro avg       0.73      0.73      0.73     23303
weighted avg       0.74      0.73      0.74     23303



In [16]:
imp_lgb_df = pd.DataFrame({'col':lgbm.feature_name_, 'imp':lgbm.feature_importances_})
imp_lgb_df = imp_lgb_df.sort_values(by='imp',ascending=False)
imp_lgb_df.reset_index(drop=True, inplace=True)
imp_lgb_df.head(15)

,col,imp
0,days_after_release,518
1,Indie_y,400
2,initialprice,344
3,achievements,296
4,price,288
5,All_time_peak,286
6,Singleplayer,223
7,dlc,222
8,num_lang,215
9,Action_y,169
